In [35]:
# modules
import random
import string
import re
from pathlib import Path
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import callbacks

import pandas as pd
from tqdm import tqdm

import gensim
import numpy as np
import pickle

import pyarabic.araby as araby
from pyarabic.araby import strip_tashkeel, strip_tatweel

### Data Preprocessing 

In [6]:
ar = pd.read_table('ArabicNewData.txt', delimiter='\\n', names=['ar'])
en = pd.read_table('EnglishNewData.txt', delimiter='\\n', names=['en'])

en['ar'] = ar['ar']
df = en.copy()
df = df.iloc[:35118]

/home/kelwa/anaconda3/envs/colab/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/home/kelwa/anaconda3/envs/colab/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [7]:
morphs = [strip_tashkeel, strip_tatweel]

def fix_ar(sent):
  sent = split_al_sent(sent)
  tokens = araby.tokenize(sent, morphs=morphs)
  sent = araby.normalize_hamza(' '.join(tokens), method='tasheel')
  return sent

In [8]:
def split_al(word):
    if word.startswith('ال'):
        return word[:2], word[2:]
    else: 
        return word

def split_al_sent(sent):
    ww = []
    for word in sent.split():
        out = split_al(word)
        if type(out) is tuple:
            for w in out:
                ww.append(w)
        else:
            ww.append(word)
    return ' '.join(w for w in ww)

In [9]:
df['ar'] = df.apply(lambda row: fix_ar(row.ar), axis=1)

In [10]:
text_pairs = []
for idx, row in df.iterrows():
  en, ar = row['en'], row['ar']
  ar = "[start] " + ar + " [end]"
  text_pairs.append((en, ar))

In [11]:
for idx, row in df.iterrows():
  if len(row.ar.split()) < 1:
    print(row.ar, '\n*')
    print(row.en)

In [12]:
for _ in range(2):
    print(random.choice(text_pairs))

('Mr Dances With Wolves.', '[start] Mr Dances With Wolves . [end]')
('At ease.', '[start] بهدوء [end]')


In [13]:
len(text_pairs)

35118

In [14]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) -  num_val_samples
train_pairs = text_pairs[: num_train_samples]
val_pairs = text_pairs[num_train_samples: num_train_samples + num_val_samples]

In [15]:
print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")

35118 total pairs
29851 training pairs
5267 validation pairs


#### Vectorizing the text data 

In [16]:
strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")


vocab_size = 20000
sequence_length = 50
batch_size = 265

def custom_standardization(input_string):
    return tf.strings.regex_replace(input_string, "[%s]" % re.escape(strip_chars), "")

eng_vectorization = TextVectorization(
            # max_tokens=vocab_size, 
            output_mode='int', 
            output_sequence_length=sequence_length)

ar_vectorization = TextVectorization(
    # max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
    max_tokens=vocab_size)

eng_texts = [pair[0] for pair in text_pairs]
ar_texts = [pair[1] for pair in text_pairs]
eng_vectorization.adapt(eng_texts)
ar_vectorization.adapt(ar_texts)

In [17]:
len(ar_vectorization.get_vocabulary()), len(eng_vectorization.get_vocabulary())

(20000, 13164)

In [19]:
def format_dataset(eng, ar):
    eng = eng_vectorization(eng)
    ar = ar_vectorization(ar)
    return ({"encoder_inputs": eng, "decoder_inputs": ar[:, :-1],}, ar[:, 1:])


def make_dataset(pairs):
    eng_texts, ar_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    ar_texts = list(ar_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, ar_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()

In [20]:
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [21]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (265, 50)
inputs["decoder_inputs"].shape: (265, 50)
targets.shape: (265, 50)


### Building the Model 

In [22]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):

      config = super().get_config().copy()
      config.update({
          'embed_dim': self.embed_dim,
          'dense_dim': self.dense_dim,
          'num_heads': self.num_heads,
      })
      return config

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, pretrained=False, weights=False, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        if not pretrained:
          self.token_embeddings = layers.Embedding(
              input_dim=vocab_size, output_dim=embed_dim
          )
        else:
          # pre-trained
          self.token_embeddings = layers.Embedding(
              input_dim=vocab_size, output_dim=embed_dim, weights=[weights]
          ) 

        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
      
    def get_config(self):

      config = super().get_config().copy()
      config.update({
      'sequence_length': self.sequence_length,
      'vocab_size': self.vocab_size,
      'embed_dim': self.embed_dim,
      })
      return config

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True


    def get_config(self):

      config = super().get_config().copy()
      config.update({
      'embed_dim': self.embed_dim,
      'latent_dim': self.latent_dim,
      'num_heads': self.num_heads,
      })
      return config

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [23]:
def get_weights(vectorizer, embeddings_path, is_gensim=False, is_local=False):
    path_to_glove_file = embeddings_path
    
    if is_gensim:
        embeddings_index = gensim.models.Word2Vec.load(embeddings_path)

    elif is_local:
        with open(embeddings_path, "rb") as f:
            embeddings_index = pickle.load(f)
    else:
        embeddings_index = {}
        with open(path_to_glove_file) as f:
            for line in f:
                word, coefs = line.split(maxsplit=1)
                coefs = np.fromstring(coefs, "f", sep=" ")
                embeddings_index[word] = coefs
    
    if not is_gensim:
        print("Found %s word vectors." % len(embeddings_index))
    else:
        print("Found %s word vectors." % (embeddings_index.wv.vectors.shape[0]))

    
    
    vocab = vectorizer.get_vocabulary()
    word_index = dict(zip(vocab, range(len(vocab))))
    num_tokens = len(vocab)
    embedding_dim = 300
    hits = 0
    misses = 0

    # Prepare embedding matrix
    embedding_matrix = np.zeros((num_tokens, embedding_dim))
    for word, i in word_index.items():
        if not is_gensim:
            embedding_vector = embeddings_index.get(word)
        else:
            if word in embeddings_index.wv:
                embedding_vector = embeddings_index.wv[word]
            else:
                embedding_vector = None
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            # This includes the representation for "padding" and "OOV"
            embedding_matrix[i] = embedding_vector
            hits += 1
        else:
            embedding_matrix[i] = np.random.uniform(-.1, .1, size=(embedding_dim))
            misses += 1
    print("Converted %d words (%d misses)" % (hits, misses))
    return embedding_matrix

In [25]:
english_embeddings = get_weights(eng_vectorization, './pretrained_en.pkl', is_local=True)

Found 10002 word vectors.
Converted 5625 words (7539 misses)


In [26]:
arabic_embeddings = get_weights(ar_vectorization, './pretrained_ar.pkl', is_local=True)

Found 10002 word vectors.
Converted 4285 words (15715 misses)


In [27]:
ar_vocab_size = len(ar_vectorization.get_vocabulary())
en_vocab_size = len(eng_vectorization.get_vocabulary())
en_vocab_size, ar_vocab_size

(13164, 20000)

In [36]:
embed_dim = 300
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, en_vocab_size, embed_dim, pretrained=True, weights=english_embeddings)(encoder_inputs)
# x = PositionalEmbedding(sequence_length, num_tokens, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, ar_vocab_size, embed_dim, pretrained=True, weights=arabic_embeddings)(decoder_inputs)
# x = PositionalEmbedding(sequence_length, ar_vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(ar_vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [43]:
googledrive_path = './our_pretrained'

In [44]:
early_stopping_cb = callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=True)
checkpoint_cb = callbacks.ModelCheckpoint(googledrive_path+'/weights_adam.ckpt', monitor='val_accuracy', save_weights_only=True,verbose=True, save_best_only=True)
tensorboard_callback = callbacks.TensorBoard(log_dir=googledrive_path+"/logs")
lr_schr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=True, factor=0.3, min_lr=0.0001)
cbs = [early_stopping_cb, checkpoint_cb, tensorboard_callback, lr_schr]

In [46]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [49]:
!nvidia-smi

Wed Jun  9 00:34:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.31       Driver Version: 465.31       CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| N/A   53C    P8     6W /  N/A |     47MiB /  6078MiB |     31%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [45]:
epochs = 100  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=cbs)

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
positional_embedding_2 (Positio (None, None, 300)    3964200     encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
transformer_encoder_1 (Transfor (None, None, 300)    4119848     positional_embedding_2[0][0]     
________________________________________________________________________________________

KeyboardInterrupt: 

In [ ]:
latest = tf.train.latest_checkpoint(googledrive_path)
transformer.load_weights(latest)

In [ ]:
ar_vocab = ar_vectorization.get_vocabulary()
ar_index_lookup = dict(zip(range(len(ar_vocab)), ar_vocab))
max_decoded_sentence_length = sequence_length


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = ar_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = ar_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in val_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts[:30])
    translated = decode_sequence(input_sentence)
    print(input_sentence, '\n', translated)
    print('*'*50)

This was originally just a water container. 
 [start] هذه كانت ال حجرة سفينة فقط [UNK] [end]
**************************************************
Room 1 4. Next to the solarium. 
 [start] ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة [end]
**************************************************
Maybe not as thick as the ones that Joshua blew down with his trumpet. 
 [start] ربما لا بين ال ثلج كما [UNK] [end]
**************************************************
Room 1 4. Next to the solarium. 
 [start] ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة ال حلقة [end]
**************************************************
You're not Han Na, are you? 
 [start] انت لست هان نا ، اليس كذلك ؟ [end]
**************************************************
I Won't Be Here,And- 
 [start] لن اكون ساكون [end]
**************************************************
You're not Han Na, are you? 
 [start] انت لست هان نا ، اليس كذلك ؟ [end]
****************

In [ ]:
def get_bleu():
  
  preds, src = [], []

  with tqdm(total=len(val_pairs), position=0, leave=True) as pbar:
    for en_sent, ar_sent in tqdm(val_pairs, position=0, leave=True):
      translated = decode_sequence(en_sent)
      preds.append(translated)
      src.append(ar_sent)
      pbar.update()

    return src, preds
    # print_scores(src, preds)



In [ ]:
def print_scores(trgs, preds):
    print('----- Bleu-n Scores -----')
    print("1:", corpus_bleu(trgs, preds, weights=[1.0/1.0])*100)
    print("2:", corpus_bleu(trgs, preds, weights=[1.0/2.0, 1.0/2.0])*100)
    print("3:", corpus_bleu(trgs, preds, weights=[1.0/3.0, 1.0/3.0, 1.0/3.0])*100)
    print("4:", corpus_bleu(trgs, preds)*100)
    print('-'*25)

In [ ]:
src, preds = get_bleu()

100%|██████████| 5267/5267 [18:20<00:00,  4.79it/s]


In [ ]:
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
print_scores(preds, src)

----- Bleu-n Scores -----
1: 40.14637653331428


/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


2: 63.3611683393814
3: 73.77039652394956
4: 79.59972885593355
-------------------------


In [ ]:
for _ in range(10):
  i = random.randint(0, 500)
  print("prediction:", preds[i][7:-6])
  print("source:", src[i][7:-6])
  print('_'*100)

prediction:  اذا كان [UNK] على ال يدي
source:  لو وق بين يدى
____________________________________________________________________________________________________
prediction:  ماذا قلت ؟
source:  - ماذا قلت ؟
____________________________________________________________________________________________________
prediction:  يبدو انك [UNK]
source:  . واو .... رييس انت تكسب ال كثير من ال اموال
____________________________________________________________________________________________________
prediction:  لا ، لا ، لا
source:  لا يوجد رد
____________________________________________________________________________________________________
prediction:  هذه هي ال احذية لكن ، اعتقد انه في سريرك
source:  هذه امتعته ، لكن اعتقد هو في حجرته .
____________________________________________________________________________________________________
prediction:  لانى [UNK]
source:  #، هذا جزيي ال مفضل لانكم #
___________________________________________________________________________________________________

In [ ]:
from PyRouge.pyrouge import Rouge

r = Rouge()

system_generated_summary = "The Kyrgyz President pushed through the law requiring the use of ink during the upcoming Parliamentary and Presidential elections In an effort to live up to its reputation in the 1990s as an island of democracy. The use of ink is one part of a general effort to show commitment towards more open elections. improper use of this type of ink can cause additional problems as the elections in Afghanistan showed. The use of ink and readers by itself is not a panacea for election ills."

manual_summmary = "The use of invisible ink and ultraviolet readers in the elections of the Kyrgyz Republic which is a small, mountainous state of the former Soviet republic, causing both worries and guarded optimism among different sectors of the population. Though the actual technology behind the ink is not complicated, the presence of ultraviolet light (of the kind used to verify money) causes the ink to glow with a neon yellow light. But, this use of the new technology has caused a lot of problems. "

[precision, recall, f_score] = r.rouge_l([system_generated_summary], [manual_summmary])

print("Precision is :"+str(precision)+"\nRecall is :"+str(recall)+"\nF Score is :"+str(f_score))

#output
"""
Precision is :0.446058091286
Recall is :0.439672801636
F Score is :0.442843380487